<a href="https://colab.research.google.com/github/sargyri/Drop_Lev/blob/master/Machine_learning/Theory_experiment_mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Lev Droplet 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/basic_regression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/tutorials/keras/basic_regression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/r2.0rc/site/en/tutorials/keras/basic_regression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Determination of ST by NN using coords (rho,theta, volume, volt)

In [ ]:
# Use seaborn for pairplot
#!pip install seaborn
#!pip install talos
#!pip install -q  --no-deps tensorflow-addons~=0.7

In [ ]:
!pip install lmfit
!pip install tensorflow_addons

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib
import pprint
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow_addons as tfa

print(tf.__version__)


Instructions for updating:
non-resource variables are not supported in the long term
2.4.1


Run the next code box only if google drive is not mounted to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


If it **is** mounted continue from here on

Check the connection at google's **GPU**

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


**Or** check the connection at google's **TPU**

In [ ]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

### Get the data
First download the dataset!

**Option 1**: Upload files

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
csvfilename=str('TritonX100_conci_0.0100_all.csv')
input = pd.read_csv(io.BytesIO(uploaded[csvfilename]))
#input = pd.read_csv(csvfilename, sep="\t")
# Dataset is now stored in a Pandas Dataframe
input.shape

**Option 2**: Import files from drive

In [ ]:
#From the drive folder that is mounted to collab go to the file that you are interested in, 
#right click and copy the path of the file. Paste it in csvfilename:
path=str('/content/drive/My Drive/DropLev/Levitator_2/Training/')

fileSDS1=pd.read_csv(str(path+'SDS/SDS_conci_0.01054_all.csv'), sep="\t")
fileSDS2=pd.read_csv(str(path+'SDS/SDS_conci_0.02108_all.csv'), sep='\t')
fileSDS3=pd.read_csv(str(path+'SDS/SDS_conci_0.04273_all.csv'), sep='\t')
fileSDS4=pd.read_csv(str(path+'SDS/SDS_conci_0.08828_all.csv'), sep="\t")
fileSDS5=pd.read_csv(str(path+'SDS/SDS_conci_0.25000_all.csv'), sep="\t")
fileSDS6=pd.read_csv(str(path+'SDS/SDS_conci_0.50000_all.csv'), sep="\t")
fileSDS7=pd.read_csv(str(path+'SDS/SDS_conci_0.75000_all.csv'), sep="\t")
fileSDS8=pd.read_csv(str(path+'SDS/SDS_conci_1.00000_all.csv'), sep="\t")
fileSDS9=pd.read_csv(str(path+'SDS/SDS_conci_1.25000_all.csv'), sep="\t")
fileSDS10=pd.read_csv(str(path+'SDS/SDS_conci_1.50000_all.csv'), sep="\t")
fileSDS11=pd.read_csv(str(path+'SDS/SDS_conci_2.00000_all.csv'), sep="\t")
fileSDS12=pd.read_csv(str(path+'SDS/SDS_conci_2.50000_all.csv'), sep="\t")
fileSDS13=pd.read_csv(str(path+'SDS/SDS_conci_3.00000_all.csv'), sep="\t")
fileSDS14=pd.read_csv(str(path+'SDS/SDS_conci_3.50000_all.csv'), sep="\t")
fileSDS15=pd.read_csv(str(path+'SDS/SDS_conci_4.00000_all.csv'), sep="\t")
fileSDS16=pd.read_csv(str(path+'SDS/SDS_conci_4.50000_all.csv'), sep="\t")
fileSDS17=pd.read_csv(str(path+'SDS/SDS_conci_5.50000_all.csv'), sep="\t")
fileSDS18=pd.read_csv(str(path+'SDS/SDS_conci_6.50000_all.csv'), sep="\t")
fileSDS19=pd.read_csv(str(path+'SDS/SDS_conci_7.50000_all.csv'), sep="\t")
fileSDS20=pd.read_csv(str(path+'SDS/SDS_conci_8.50000_all.csv'), sep="\t")
fileSDS21=pd.read_csv(str(path+'SDS/SDS_conci_24.75900_all.csv'), sep="\t")

fileTrit1=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.00250_all.csv'), sep="\t")
fileTrit2=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.00500_all.csv'), sep="\t")
fileTrit3=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.00750_all.csv'), sep="\t")
fileTrit4=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.01000_all.csv'), sep="\t")
fileTrit5=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.01500_all.csv'), sep="\t")
fileTrit6=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.02000_all.csv'), sep="\t")
fileTrit7=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.02500_all.csv'), sep="\t")
fileTrit8=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.03500_all.csv'), sep="\t")
fileTrit9=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.05000_all.csv'), sep="\t")
fileTrit10=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.07500_all.csv'), sep="\t")
fileTrit11=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.10000_all.csv'), sep="\t")
fileTrit12=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.12500_all.csv'), sep="\t")
fileTrit13=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.15000_all.csv'), sep="\t")
fileTrit14=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.17500_all.csv'), sep="\t")
fileTrit15=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.20000_all.csv'), sep="\t")
fileTrit16=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.25000_all.csv'), sep="\t")
fileTrit17=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.30000_all.csv'), sep="\t")
fileTrit18=pd.read_csv(str(path+'TritonX100/TritonX_conci_0.45000_all.csv'), sep="\t")
fileTrit19=pd.read_csv(str(path+'TritonX100/TritonX_conci_1.00000_all.csv'), sep="\t")
fileTrit20=pd.read_csv(str(path+'TritonX100/TritonX_conci_3.00000_all.csv'), sep="\t")
fileTrit21=pd.read_csv(str(path+'TritonX100/TritonX_conci_10.00000_all.csv'), sep="\t")

fileCTAB1=pd.read_csv(str(path+'CTAB/CTAB_conci_0.00100_all.csv'), sep="\t")
fileCTAB2=pd.read_csv(str(path+'CTAB/CTAB_conci_0.00200_all.csv'), sep="\t")
fileCTAB3=pd.read_csv(str(path+'CTAB/CTAB_conci_0.00500_all.csv'), sep="\t")
fileCTAB4=pd.read_csv(str(path+'CTAB/CTAB_conci_0.01000_all.csv'), sep="\t")
fileCTAB5=pd.read_csv(str(path+'CTAB/CTAB_conci_0.01500_all.csv'), sep="\t")
fileCTAB6=pd.read_csv(str(path+'CTAB/CTAB_conci_0.05500_all.csv'), sep="\t")
fileCTAB7=pd.read_csv(str(path+'CTAB/CTAB_conci_0.10000_all.csv'), sep="\t")
fileCTAB8=pd.read_csv(str(path+'CTAB/CTAB_conci_0.15000_all.csv'), sep="\t")
fileCTAB9=pd.read_csv(str(path+'CTAB/CTAB_conci_0.20000_all.csv'), sep="\t")
fileCTAB10=pd.read_csv(str(path+'CTAB/CTAB_conci_0.25000_all.csv'), sep="\t")
fileCTAB11=pd.read_csv(str(path+'CTAB/CTAB_conci_0.30000_all.csv'), sep="\t")
fileCTAB12=pd.read_csv(str(path+'CTAB/CTAB_conci_0.40000_all.csv'), sep="\t")
fileCTAB13=pd.read_csv(str(path+'CTAB/CTAB_conci_0.50000_all.csv'), sep="\t")
fileCTAB14=pd.read_csv(str(path+'CTAB/CTAB_conci_0.60000_all.csv'), sep="\t")
fileCTAB15=pd.read_csv(str(path+'CTAB/CTAB_conci_0.70000_all.csv'), sep="\t")
fileCTAB16=pd.read_csv(str(path+'CTAB/CTAB_conci_0.80000_all.csv'), sep="\t")
fileCTAB17=pd.read_csv(str(path+'CTAB/CTAB_conci_0.85000_all.csv'), sep="\t")
fileCTAB18=pd.read_csv(str(path+'CTAB/CTAB_conci_0.90000_all.csv'), sep="\t")
fileCTAB19=pd.read_csv(str(path+'CTAB/CTAB_conci_1.00000_all.csv'), sep="\t")
fileCTAB20=pd.read_csv(str(path+'CTAB/CTAB_conci_5.00000_all.csv'), sep="\t")
fileCTAB21=pd.read_csv(str(path+'CTAB/CTAB_conci_13.15224_all.csv'), sep="\t")

#input=np.vstack([fileSDS1, fileSDS2, fileSDS3, fileSDS4, fileSDS5, fileSDS6, fileSDS7, fileSDS8, fileSDS9, fileSDS10, fileSDS11, 
                 #fileSDS12, fileSDS13, fileSDS14, fileSDS15, fileSDS16, fileSDS17, fileSDS18, fileSDS19, fileSDS20, fileSDS21,
                # fileTrit1, fileTrit2, fileTrit3, fileTrit4, fileTrit5, fileTrit6, fileTrit7, fileTrit8, fileTrit9, fileTrit10, fileTrit11,
                # fileTrit12, fileTrit13, fileTrit14, fileTrit15, fileTrit16, fileTrit17, fileTrit18, fileTrit19, fileTrit20, fileTrit21,
                # fileCTAB1, fileCTAB2, fileCTAB3, fileCTAB4, fileCTAB5, fileCTAB6, fileCTAB7, fileCTAB8, fileCTAB9, fileCTAB10, fileCTAB11,
                # fileCTAB12, fileCTAB13, fileCTAB14, fileCTAB15, fileCTAB16, fileCTAB17, fileCTAB18, fileCTAB19, fileCTAB20, fileCTAB21])
# input=np.vstack([fileSDS5, fileSDS6, fileSDS7, fileSDS8, fileSDS9, fileSDS10, fileSDS13, fileSDS14,
#                  fileTrit7, fileTrit8, fileTrit9, fileTrit10, fileTrit11,
#                  fileCTAB9, fileCTAB10, fileCTAB11, fileCTAB12,fileCTAB13, fileCTAB14])
# input.shape

#csvfilename=str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_1.50000_ML_data.csv')

#input = pd.read_csv(csvfilename)

In [ ]:
# input=np.vstack([fileSDS1, fileSDS2, fileSDS3, fileSDS4, fileSDS5, fileSDS6, fileSDS7,fileSDS8, fileSDS9, fileSDS10, fileSDS11, 
#                  fileSDS12, fileSDS13, fileSDS14, fileSDS15, fileSDS16, fileSDS17, fileSDS18,fileSDS19, fileSDS20, fileSDS21,
#                  fileTrit1, fileTrit2, fileTrit3, fileTrit4, fileTrit5,  fileTrit6, fileTrit7, fileTrit8, fileTrit9, fileTrit10, fileTrit11,
#                  fileTrit12, fileTrit13, fileTrit14, fileTrit15, fileTrit16, fileTrit17, fileTrit18, fileTrit19, fileTrit20, fileTrit21,
#                  fileCTAB1, fileCTAB2, fileCTAB3, fileCTAB4, fileCTAB5,fileCTAB6, fileCTAB7, fileCTAB8, fileCTAB9, fileCTAB10, fileCTAB11,
#                  fileCTAB12, fileCTAB13, fileCTAB14, fileCTAB15, fileCTAB16, fileCTAB17, fileCTAB18, fileCTAB19, fileCTAB20, fileCTAB21])

# input=np.vstack([fileSDS1, fileSDS2, fileSDS3, fileSDS4, fileSDS5, fileSDS6, fileSDS7, fileSDS19, fileSDS20, fileSDS21,
#                  fileTrit1, fileTrit2, fileTrit3, fileTrit4, fileTrit5, fileTrit17, fileTrit18, fileTrit19, fileTrit20, fileTrit21,
#                  fileCTAB1, fileCTAB2, fileCTAB3, fileCTAB4, fileCTAB5, fileCTAB15, fileCTAB16, fileCTAB17, fileCTAB18, fileCTAB19, fileCTAB20, fileCTAB21])

input=np.vstack([fileSDS8, fileSDS9, fileSDS10, fileSDS11, fileSDS12, fileSDS13, fileSDS14, fileSDS15, fileSDS16, fileSDS17, fileSDS18,
                 fileTrit6, fileTrit7, fileTrit8, fileTrit9, fileTrit10, fileTrit11, fileTrit12, fileTrit13, fileTrit14, fileTrit15, fileTrit16,
                 fileCTAB6, fileCTAB7, fileCTAB8, fileCTAB9, fileCTAB10, fileCTAB11, fileCTAB12, fileCTAB13, fileCTAB14])

# input=np.vstack([fileSDS8, fileSDS9, fileSDS10, fileSDS11, fileSDS12, fileSDS13,  fileSDS14, fileSDS15, fileSDS16, fileSDS17, 
#            fileTrit7, fileTrit8, fileTrit9, fileTrit10, fileTrit11,
#            fileTrit12, fileTrit13, fileTrit14, fileTrit15,  
#            fileCTAB7,fileCTAB8, fileCTAB9, fileCTAB10, fileCTAB11, 
#            fileCTAB13])

# input=fileSDS8


In [ ]:
column_names=[]

for i in range(300):
  column_names.append('rho_%d' %i)
for i in range(300):
  column_names.append('phi_%d' %i)
for i in range(300):
  column_names.append('theta_%d' %i)
column_names.append('height')  #900
column_names.append('width')  #901
column_names.append('volume')#902
column_names.append('R_sph')#903
column_names.append('volt')#904
column_names.append('Intensity')#905
column_names.append('time') #906
column_names.append('Power')#907
#column_names.append('Ps')
#column_names.append('Ps_err')
column_names.append('centre_x') #908
column_names.append('centre_y') #909
column_names.append('surfactant_conc') #910
column_names.append('Aspect_Ratio') #911
column_names.append('st') #912


#raw_dataset = pd.read_csv(csvfilename, sep="\t")
#raw_dataset.columns=column_names


raw_dataset = pd.DataFrame(data=input,  columns=column_names) 
              
raw_dataset.shape

dataset = raw_dataset.copy()

dataset.centre_y=-dataset.centre_y

# for i in range(300):
#  dataset.pop('rho_%d' %i)
for i in range(300):
  dataset.pop('phi_%d' %i)
# for i in range(300):
#  dataset.pop('theta_%d' %i)
dataset.pop("height")
dataset.pop("width")
# dataset.pop("volume")
# dataset.pop("R_sph")
dataset.pop('volt')
dataset.pop('Intensity')
dataset.pop('time')
dataset.pop('Power')
# dataset.pop('Ps')
#dataset.pop('Ps_err')
dataset.pop("centre_x")
dataset.pop("centre_y")
dataset.pop('surfactant_conc')
dataset.pop("Aspect_Ratio")


dataset.tail()

#We keep: volume	Power	centre_y	Aspect_Ratio	st

,rho_0,rho_1,rho_2,rho_3,rho_4,rho_5,rho_6,rho_7,rho_8,rho_9,rho_10,rho_11,rho_12,rho_13,rho_14,rho_15,rho_16,rho_17,rho_18,rho_19,rho_20,rho_21,rho_22,rho_23,rho_24,rho_25,rho_26,rho_27,rho_28,rho_29,rho_30,rho_31,rho_32,rho_33,rho_34,rho_35,rho_36,rho_37,rho_38,rho_39,...,theta_262,theta_263,theta_264,theta_265,theta_266,theta_267,theta_268,theta_269,theta_270,theta_271,theta_272,theta_273,theta_274,theta_275,theta_276,theta_277,theta_278,theta_279,theta_280,theta_281,theta_282,theta_283,theta_284,theta_285,theta_286,theta_287,theta_288,theta_289,theta_290,theta_291,theta_292,theta_293,theta_294,theta_295,theta_296,theta_297,theta_298,theta_299,volume,st
156655,0.822386,0.823070,0.823239,0.825227,0.825555,0.824092,0.821238,0.818078,0.817721,0.815634,0.813631,0.813586,0.810163,0.808581,0.807087,0.805554,0.797752,0.781719,0.775103,0.769040,0.763541,0.762256,0.759796,0.758620,0.733620,0.722741,0.716765,0.711439,0.708936,0.704215,0.689382,0.688479,0.680292,0.679033,0.674743,0.671057,0.664661,0.662158,0.660377,0.652095,...,-0.827844,-0.920926,-0.924124,-0.956637,-0.971200,-0.978452,-0.988549,-1.017115,-1.024199,-1.079945,-1.151564,-1.158102,-1.216600,-1.222858,-1.233662,-1.239844,-1.244438,-1.261278,-1.265890,-1.282570,-1.309578,-1.335066,-1.350062,-1.355728,-1.410284,-1.429563,-1.434211,-1.462728,-1.472522,-1.481830,-1.486882,-1.500817,-1.514780,-1.542897,-1.552194,-1.556844,-1.561494,-1.570796,1.764766,38.500904
156656,0.822386,0.823070,0.823239,0.821416,0.821745,0.820293,0.817461,0.818078,0.813982,0.811903,0.813631,0.809883,0.810163,0.808581,0.807087,0.801922,0.797752,0.778296,0.775103,0.769040,0.763541,0.762256,0.759796,0.758620,0.733620,0.722741,0.716765,0.711439,0.708936,0.704215,0.689382,0.685423,0.680292,0.679033,0.674743,0.671057,0.664661,0.662158,0.660377,0.652095,...,-0.816396,-0.910221,-0.917701,-0.950547,-0.960931,-0.968239,-0.982794,-1.007249,-1.014392,-1.077545,-1.149534,-1.156107,-1.214898,-1.221189,-1.232046,-1.238260,-1.242873,-1.259798,-1.264429,-1.281194,-1.308337,-1.333948,-1.349015,-1.354711,-1.409530,-1.428899,-1.433569,-1.462728,-1.472062,-1.481830,-1.486490,-1.500490,-1.514780,-1.542766,-1.552194,-1.556844,-1.561494,-1.570796,1.762896,38.499712
156657,0.822573,0.822786,0.822919,0.823070,0.823426,0.823852,0.822092,0.822457,0.823657,0.822217,0.818954,0.818713,0.813982,0.814799,0.809142,0.807522,0.805990,0.803189,0.800745,0.799657,0.798659,0.794785,0.785676,0.764862,0.759796,0.753192,0.748320,0.745215,0.743564,0.742796,0.716970,0.706155,0.701666,0.694930,0.692980,0.689382,0.674710,0.674743,0.667790,0.667122,...,-0.741947,-0.757954,-0.808650,-0.861586,-0.887701,-0.902728,-0.913481,-0.992894,-1.017115,-1.024199,-1.038292,-1.045301,-1.059240,-1.086790,-1.093606,-1.151564,-1.216600,-1.276562,-1.282570,-1.293157,-1.299080,-1.303703,-1.320031,-1.324659,-1.339706,-1.365013,-1.390097,-1.404872,-1.409530,-1.462728,-1.481830,-1.486490,-1.514780,-1.524101,-1.533604,-1.538250,-1.552194,-1.566145,1.768315,38.503163
156658,0.816622,0.817186,0.817338,0.817902,0.819198,0.815699,0.817955,0.814972,0.815573,0.811311,0.809198,0.807171,0.807080,0.803610,0.802007,0.804156,0.798909,0.794612,0.778308,0.775011,0.768833,0.761904,0.760626,0.758179,0.757011,0.732146,0.721336,0.715399,0.710114,0.707627,0.702939,0.691228,0.687334,0.682313,0.681101,0.673345,0.667048,0.664653,0.662890,0.654785,...,-0.829685,-0.841003,-0.915101,-0.940452,-0.947797,-0.962433,-0.969723,-0.979906,-1.008631,-1.015757,-1.071856,-1.143962,-1.209439,-1.214041,-1.226621,-1.231232,-1.237463,-1.254426,-1.259053,-1.275859,-1.301789,-1.328725,-1.343818,-1.348488,-1.403695,-1.423197,-1.428565,-1.457272,-1.467150,-1.476514,-1.481202,-1.495644,-1.509696,-1.538021,-1.547381,-1.552063,-1.556746,-1.566113,1.753342,38.493602
156659,0.816649,0.817338,0.817508,0.818125,0.815699,0.816030,0.814595,0.811797,0.812419,0.808375,0.806309,0.808049,0.804329,0.804638,0.799403,0.801596,0.796479,0.792391,0.773169,0.770040,0.764051,0.758632,0.757368,0.754950,0.753796,0.729203,0.718531,0.712

### Selectively choose data with (almost) constant volume

In [ ]:
# new_input=[]
# vol_min=2.99
# vol_max=3.01
# for i in range(len(dataset)):
#   if dataset['volume'].iloc[i] >= vol_min and dataset['volume'].iloc[i] <= vol_max:
#     vol_ct_data=dataset.iloc[i, :]
#     new_input.append(vol_ct_data)
# new_input=np.asarray(new_input)
# new_input.shape

In [ ]:
# column_names=[]

# for i in range(300):
#   column_names.append('rho_%d' %i)
# for i in range(300):
#   column_names.append('theta_%d' %i)
# column_names.append('volume')
# column_names.append('R_sph')
# column_names.append('volt')
# column_names.append('st')

# new_dataset=pd.DataFrame(new_input, columns=column_names)
# new_dataset.pop('R_sph')
# new_dataset.tail()

In [ ]:
# plt.figure()
# sns.distplot(new_dataset['volt'])
# plt.xlabel('Voltage [V]')
# plt.ylabel('Count')

# plt.figure()
# sns.distplot(new_dataset['st'])
# plt.xlabel('Surface tension [mN/m]')
# plt.ylabel('Count')

In [ ]:
# #Drop volume if it's constant
# new_dataset.pop('volume')

### Calculate the acoustic pressure from the simple model

In [ ]:
new_input=np.asarray(dataset)

In [ ]:
# If no contour points were droped
n_ST=603
n_Rsph=602
rho_i=0
rho_f=300
theta_i=300
theta_f=600

In [ ]:
from lmfit import Model, fit_report, Parameters

def model_fit(th, Ps):
    """
    Express the model that will be used to fit the experimental data (simple approach).

    Input parameters
    ----------
    dB:     deciBell
    th:     angle theta
    Returns
    -------
    Expression of fitting model.

    """
    rho=-((3/(64*gamma))*R_sph**2*Ps**2*Cg_air*(1+((7/5)*(k_o*R_sph)**2)))*(3*(np.cos(th))**2-1)+R_sph
    return rho


data_Ps=[]
Cg_air=1/101325 #Pa**(-1)
k_o=2*np.pi*40/340 #mm^-1

for i in range(len(new_input)):
  gamma=new_input[i, n_ST]
  R_sph=new_input[i, n_Rsph]
  rho=new_input[i, rho_s, rho_e]
  gmodel = Model(model_fit, nan_policy='omit')
    
  fit_param = Parameters()
  fit_param.add('Ps', value=700, min=10, max=4000)
  
  result = gmodel.fit(rho, th=new_input[i, theta_i:theta_f], params=fit_param)         #rho_final, th: x and y to be evaluated, respectively
  Ps=result.params['Ps'].value
  data_Ps.append(Ps)
data_Ps=np.asarray(data_Ps)

plt.figure()
plt.plot(data_Ps)
plt.xlabel('Image')
plt.ylabel('Acoustic pressure [Pa]')

#### Check the fitting of the simple model

In [ ]:
i=100
plt.figure()
plt.plot(new_input[i, 300:600], rho, 'bo')
plt.plot(new_input[i, 300:600], result.best_fit, 'r-', label='best fit')

In [ ]:
column_names=[]

for i in range(300):
  column_names.append('rho_%d' %i)
for i in range(300):
  column_names.append('theta_%d' %i)
column_names.append('volume')
column_names.append('R_sph')
column_names.append('volt')
column_names.append('st')

new_dataset=pd.DataFrame(new_input, columns=column_names)

In [ ]:
# #Add the calclulated Ps values to the new_dataset dataframe
new_dataset['Ps']=data_Ps

#Drop the features that we don't have in the saved model
new_dataset.pop('R_sph')
new_dataset.pop('volt')
# new_dataset.pop('volume')
new_dataset.tail()

### Drop contour points

In [ ]:
dataset=new_dataset.copy()

# If we want to drop 150 datapoints then n= 299 

# If we want to drop 150 datapoints then n= 250

n=250
for i in range(0, n, 2):
  dataset.pop('theta_%d' %i)

for i in range(0,n, 2):
  dataset.pop('rho_%d' %i)
dataset.tail()

In [ ]:
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

#Check that the drop contour is still well defined!

# For 150 contour datapoints:
# plt.figure()
# plt.plot(dataset.iloc[600, 150:300], dataset.iloc[600, 0:150])


# For 175 contour datapoints:
n=200
plt.figure()
plt.plot(dataset.iloc[n, 175:350], dataset.iloc[n, 0:175], '-o')

x, y=pol2cart(dataset.iloc[n, 0:175].values, dataset.iloc[n, 175:350].values)

plt.figure()
plt.plot(x, y, '-o')

### Keep Ps values within a specific range

In [ ]:
# newer_input=[]
# Ps_min=1500
# Ps_max=2100
# for i in range(len(new_dataset)):
#   if new_dataset['Ps'].iloc[i] >= Ps_min and new_dataset['Ps'].iloc[i] <= Ps_max:
#     Ps_range=new_dataset.iloc[i, :]
#     newer_input.append(Ps_range)
# newer_input=np.asarray(newer_input)

# newer_input.shape

In [ ]:
# plt.figure()
# plt.plot(newer_input[:, 601])
# plt.ylabel('Acoustic pressure [Pa]')
# plt.xlabel('droplet')

### Store values in new DataFrame

In [ ]:
# column_names=[]

# for i in range(300):
#   column_names.append('rho_%d' %i)
# for i in range(300):
#   column_names.append('theta_%d' %i)
# # column_names.append('volume')
# column_names.append('st')
# column_names.append('Ps')

# new_df=pd.DataFrame(newer_input, columns=column_names)

# #Drop ST:
# new_df.pop('st')

### Split the data into train and test

Now split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our model.

In [ ]:
# train_dataset = dataset.sample(frac=0.8,random_state=1)
# test_dataset = dataset.drop(train_dataset.index)

Also look at the overall statistics:

In [ ]:
# # dataset.pop('st')

# dataset_name=dataset.iloc[80000:90000]

# stats_input_df=dataset_name.describe()
# stats_input_df=stats_input_df.transpose()
# stats_input_df.tail()

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("st")

train_stats = train_stats.transpose()
train_stats

In [ ]:
# new_dataset.iloc[:, 601]
# newer_input=np.asarray(new_dataset.iloc[80000:90000])

### Normalize values, Import model and test predictions

In [ ]:
# def norm_test(x):
#   return (x - stats_input_df['mean']) / stats_input_df['std']

# #import model from drive
# folder=str('theory_experiment_mix/')
# filename=str('All_surf_all_vary_175points_trial_1.h5')
# model = tf.keras.models.load_model('/content/drive/My Drive/DropLev/MachineLearningModels/'+folder+filename)
# model.summary()

# norm_input_array=norm_test(dataset_name)
# prediction=model.predict(norm_input_array).flatten()
# true_values=newer_input[:, 601]

# plt.figure()
# plt.xlabel('Frame number')
# plt.ylabel('Surface tension [ mN/m ]')
# plt.ylim(20, 75)
# plt.plot(prediction, label='Predicted values')
# plt.plot(true_values, label='Real values')
# plt.legend()

# plt.figure()
# plt.scatter(true_values, prediction)
# plt.title('Normalized new test data')
# plt.xlabel('True Values [surface tension - mN/m]')
# plt.ylabel('Predictions [surface tension - mN/m]')
# plt.axis('equal')
# plt.axis('square')
# plt.xlim([10,75])
# plt.ylim([10,75])
# _ = plt.plot([-100, 100], [-100, 100])
# plt.show()

# error = prediction - true_values
# plt.figure()
# plt.hist(error, bins = 50)
# plt.xlabel("Prediction Error [mN/m]")
# _ = plt.ylabel("Count")
# #plt.ylim(0, 30)
# plt.show()


### Train the model

Train the model for 1000 epochs, and record the training and validation accuracy in the `history` object.

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 55 == 0: print('')
    print('.', end='')
    


EPOCHS = 200


early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0, patience=15, verbose=0, mode='auto')


history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[early_stop, PrintDot()])  # callbacks=callbacks_list

Visualize the model's training progress using the stats stored in the `history` object.

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Let's automatically stop training when the validation score doesn't improve. We'll use an *EarlyStopping callback* that tests a training condition for  every epoch. If a set amount of epochs elapses without showing improvement, then automatically stop the training.

You can learn more about this callback [here](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping).

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [ST]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Training Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Validation Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$ST^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Training Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Validation Error')
  plt.ylim([0,40])
  plt.legend()
  plt.show()


plot_history(history)

Let's see how well the model generalizes by using the **test** set, which we did not use when training the model.  This tells us how well we can expect the model to predict when we use it in the real world.

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print("Testing set Mean Abs Error: {:5.2f} mN/m".format(mae))
# keras.metrics.accuracy(normed_test_data, test_labels)

### Make predictions

Finally, predict volume values using data in the testing set:

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.title('Normalized test data')
plt.xlabel('True Values [surface tension - mN/m]')
plt.ylabel('Predictions [surface tension - mN/m]')
plt.axis('equal')
plt.axis('square')
plt.xlim([20,75])
plt.ylim([20,75])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()

test_predictions2 = model.predict(normed_train_data).flatten()

plt.scatter(train_labels, test_predictions2)
plt.title('Normalized training data')
plt.xlabel('True Values [surface tension - mN/m]')
plt.ylabel('Predictions [surface tension - mN/m]')
plt.axis('equal')
plt.axis('square')
plt.xlim([20,75])
plt.ylim([20,75])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()


In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error [surface tension - mN/m]")
_ = plt.ylabel("Count")
# plt.xlim(-20, 20)
plt.show()

# Saving the final Model

When you are satisfied with the model and the accuracy continue from here:

Saving the model in a **SavedModel** format

In [ ]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

In [ ]:
# Save the entire model as a SavedModel.
folder=str('Experimental_surfactants/')
filename=str('TritonX_all_vary_175points_trial_2.h5')
!mkdir -p '/content/drive/My Drive/DropLev/MachineLearningModels' #Ceates a folder named "MachineLearningModels"
model.save('/content/drive/My Drive/DropLev/MachineLearningModels/'+folder+filename)  #Saves the file

The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. Inspect the saved model directory:

In [ ]:
# DropLev_st_prediction_model directory
#!ls content/drive/My Drive/DropLev/MachineLearningModels

# Contains an assets folder, saved_model.pb, and variables folder.
#!ls content/drive/My Drive/DropLev/MachineLearningModels/SDS_1.h5

Saving the model in a **HDF5** format

In [ ]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
#model.save('content/drive/My Drive/DropLev/MachineLearningModels/'+filename) 